# Evaluate Object Detection with OpenVINO IR

## Import everything

In [ ]:
import cv2
import numpy as np

from ote_sdk.configuration.helper import create as create_parameters_from_parameters_schema
from ote_sdk.entities.inference_parameters import InferenceParameters
from ote_sdk.entities.label_schema import LabelSchemaEntity
from ote_sdk.entities.model import ModelEntity
from ote_sdk.entities.resultset import ResultSetEntity
from ote_sdk.entities.subset import Subset
from ote_sdk.entities.task_environment import TaskEnvironment
from ote_sdk.usecases.tasks.interfaces.export_interface import ExportType
from ote_cli.utils.io import read_label_schema, read_model

from ote_cli.datasets import get_dataset_class
from ote_cli.registry import Registry
from ote_cli.utils.importing import get_impl_class

## Register templates

In [ ]:
templates_dir = '../../external'
registry = Registry(templates_dir)
print(registry)

## Load model template and its hyper parameters

In [ ]:
model_template = registry.get('Custom_Object_Detection_Gen3_SSD')
hyper_parameters = model_template.hyper_parameters.data
hyper_parameters = create_parameters_from_parameters_schema(hyper_parameters)

## Get dataset instantiated

In [ ]:
Dataset = get_dataset_class(model_template.task_type)
weight = '../../data/exportable_code.zip'

dataset = Dataset(
    test_subset={'ann_file': '../../data/annotations/instances_NONE.json',
                  'data_root': '../../data/fruit5/images/NONE'},
)
labels_schema = read_label_schema(weight)

## Create Task 

In [ ]:
Task = get_impl_class(model_template.entrypoints.openvino)

environment = TaskEnvironment(
    model=None,
    hyper_parameters=hyper_parameters,
    label_schema=labels_schema,
    model_template=model_template)
        
environment.model = read_model(environment.get_model_configuration(), weight, None)

task = Task(task_environment=environment)

## Evaluate quality metric

In [ ]:
validation_dataset = dataset.get_subset(Subset.TESTING)
predicted_validation_dataset = task.infer(
    validation_dataset.with_empty_annotations(),
    InferenceParameters(is_evaluation=True))

resultset = ResultSetEntity(
        model=environment.model,
        ground_truth_dataset=validation_dataset,
        prediction_dataset=predicted_validation_dataset,
    )
task.evaluate(resultset)
print(resultset.performance)


## Draw bounding boxes

In [ ]:
import IPython
import PIL

for predictions, item in zip(predicted_validation_dataset, validation_dataset.with_empty_annotations()):
    image = item.numpy.astype(np.uint8)
    for box, label in zip(predictions.annotation_scene.shapes, predictions.annotation_scene.get_labels()):
        x1 = int(box.x1 * image.shape[1])
        x2 = int(box.x2 * image.shape[1])
        y1 = int(box.y1 * image.shape[0])
        y2 = int(box.y2 * image.shape[0])
        cv2.rectangle(image, (x1, y1+5), (x2, y2), (0, 0, 255), 3)  
        cv2.putText(image, label.name, (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    IPython.display.display(PIL.Image.fromarray(image))
    break